In [20]:
# Import packages
import os
import glob
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Reproducibility
np.random.seed(478)
tf.random.set_seed(478)

In [32]:
# Read in data from .csv files
train = pd.read_csv('./Data/train.csv')
test = pd.read_csv('./Data/test.csv')

# Split data into training/validation sets based on a 70/30 split
idx = range(25200)
valid = train.iloc[~train.index.isin(idx)]
train = train.iloc[train.index.isin(idx)]

# Convert Pandas dataframes to valid Numpy arrays
def convert(X):
    data = np.empty((X.shape[0], 28, 28, 1))
    for i in range(X.shape[0]):
        img = np.array(X.iloc[i, :]).reshape((28, 28, 1))
        data[i] = img
    return(np.array(data))

# Define training set
X_train = convert(train.drop('label', axis = 1)) / 255
y_train = train['label']

# Define validation set
X_valid = convert(valid.drop('label', axis = 1)) / 255
y_valid = valid['label']

# Define testing set
X_test = convert(test) / 255

In [11]:
# Define model architecture
model = keras.Sequential([
    layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [28, 28, 1]),
    layers.MaxPool2D(),

    layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'),
    layers.MaxPool2D(),

    layers.Flatten(),
    layers.Dense(units = 10, activation = tf.nn.softmax)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                16010     
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
__________________________________________________

In [33]:
# Compile model
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

# Train model
trained = model.fit(
    X_train,
    y_train,
    epochs = 20
)

Epoch 1/20
788/788 [==============================] - 8s 9ms/step - loss: 0.5828 - accuracy: 0.8218
Epoch 2/20
788/788 [==============================] - 7s 9ms/step - loss: 0.0892 - accuracy: 0.9744
Epoch 3/20
788/788 [==============================] - 7s 9ms/step - loss: 0.0553 - accuracy: 0.9829
Epoch 4/20
788/788 [==============================] - 7s 9ms/step - loss: 0.0366 - accuracy: 0.9889
Epoch 5/20
788/788 [==============================] - 8s 10ms/step - loss: 0.0322 - accuracy: 0.9895
Epoch 6/20
788/788 [==============================] - 7s 9ms/step - loss: 0.0225 - accuracy: 0.9933
Epoch 7/20
788/788 [==============================] - 8s 10ms/step - loss: 0.0170 - accuracy: 0.9948
Epoch 8/20
788/788 [==============================] - 7s 9ms/step - loss: 0.0158 - accuracy: 0.9957
Epoch 9/20
788/788 [==============================] - 7s 9ms/step - loss: 0.0115 - accuracy: 0.9966
Epoch 10/20
788/788 [==============================] - 7s 9ms/step - loss: 0.0089 - accuracy: 0.99

In [42]:
# Calculate MSE on validation set
pred = pd.DataFrame(model.predict(X_valid)).idxmax(axis = 1)
np.mean(pred.reset_index(drop = True) == y_valid.reset_index(drop = True))

ValueError: Can only compare identically-labeled Series objects